<a href="https://colab.research.google.com/github/goru001/nlp-for-tamil/blob/master/Explore_Pretrained_Tamil_Language_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Loading a pretrained language model and exploring it.

- Model Source data: Entire text from all 127000 Tamil Wikipedia articles from March 2019.
- Model architecture: FastAI AWD_LSTM (Representation: 400 dimensional thought vector)
- Wordparts : 8000 word parts using google sentencepiece
- Perplexity: ~20 on validation data

### Step 1. Imports

In [1]:
!pip install fastai --upgrade

Requirement already up-to-date: fastai in /usr/local/lib/python3.6/dist-packages (1.0.51)


In [2]:
!pip install sentencepiece

In [0]:
from fastai.text import *
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
import sentencepiece as spm

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
import fastai, torch
fastai.__version__ , torch.__version__

('1.0.51', '1.0.1.post2')

### Step 2. Download Tamil LM models from Dropbox

In [18]:
!wget https://www.dropbox.com/s/zozzrgawulkwtxa/wikitalm_8k_447_third.pth

--2019-04-06 13:53:02--  https://www.dropbox.com/s/zozzrgawulkwtxa/wikitalm_8k_447_third.pth
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:6018:1::a27d:301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/zozzrgawulkwtxa/wikitalm_8k_447_third.pth [following]
--2019-04-06 13:53:02--  https://www.dropbox.com/s/raw/zozzrgawulkwtxa/wikitalm_8k_447_third.pth
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc02c52ea5453d59ae7b84032a4b.dl.dropboxusercontent.com/cd/0/inline/Aegt5FQ5FbvOYOUbIr8chSyC4L5dN4DqE3kIKB601ohRyjmlZBIbVlSjcMe-wbDZWrlqDDgBdoadbXQ4qR3zoSPq7zu8jAvtnbs9UVeaE12ywQ/file# [following]
--2019-04-06 13:53:02--  https://uc02c52ea5453d59ae7b84032a4b.dl.dropboxusercontent.com/cd/0/inline/Aegt5FQ5FbvOYOUbIr8chSyC4L5dN4DqE3kIKB601ohRyjmlZBIbVlSjcMe-wbDZWrlqDDgBdoadbXQ4qR3zoSPq7zu8jAvt

In [17]:
!wget https://www.dropbox.com/s/hl7cowdki4ngf65/data_lm_tawiki_8k.pkl

--2019-04-06 13:52:53--  https://www.dropbox.com/s/hl7cowdki4ngf65/data_lm_tawiki_8k.pkl
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:6018:1::a27d:301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/hl7cowdki4ngf65/data_lm_tawiki_8k.pkl [following]
--2019-04-06 13:52:54--  https://www.dropbox.com/s/raw/hl7cowdki4ngf65/data_lm_tawiki_8k.pkl
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd74ce3d5e1d7c5d44e6f515a59.dl.dropboxusercontent.com/cd/0/inline/AegEQ0aAEXfBYukJgEAPcX0EoTzq0-78KRwXSUn3GVQt_5UN6zXb98maTCjhXVqfB6jhzWb5S3lROy4_-yXnpzzAz-PTc-ZsigXjV9oVBZ1J8w/file# [following]
--2019-04-06 13:52:54--  https://ucd74ce3d5e1d7c5d44e6f515a59.dl.dropboxusercontent.com/cd/0/inline/AegEQ0aAEXfBYukJgEAPcX0EoTzq0-78KRwXSUn3GVQt_5UN6zXb98maTCjhXVqfB6jhzWb5S3lROy4_-yXnpzzAz-PTc-ZsigXjV9oVBZ1J

In [16]:
!wget https://www.dropbox.com/s/9q1pkf6c7ahs9qm/tamil_spm_8k.model

--2019-04-06 13:52:48--  https://www.dropbox.com/s/9q1pkf6c7ahs9qm/tamil_spm_8k.model
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:6018:1::a27d:301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/9q1pkf6c7ahs9qm/tamil_spm_8k.model [following]
--2019-04-06 13:52:48--  https://www.dropbox.com/s/raw/9q1pkf6c7ahs9qm/tamil_spm_8k.model
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2989d86e3def2443cd4233272c.dl.dropboxusercontent.com/cd/0/inline/AehG8uENNhmbPoXUSXwSZX-MZD44svacyvfKo6vfUJVvh2tu_raui5wRmDj1dGyCfJSM7vnXtQNstCqlfsGQqdpXAjxlKS0yC7IC3IUO3-SCWQ/file# [following]
--2019-04-06 13:52:48--  https://uc2989d86e3def2443cd4233272c.dl.dropboxusercontent.com/cd/0/inline/AehG8uENNhmbPoXUSXwSZX-MZD44svacyvfKo6vfUJVvh2tu_raui5wRmDj1dGyCfJSM7vnXtQNstCqlfsGQqdpXAjxlKS0yC7IC3IUO3-SCWQ/file
R

In [14]:
!wget https://www.dropbox.com/s/5u4l67faq46jbvw/tamil_spm_8k.vocab

--2019-04-06 13:52:31--  https://www.dropbox.com/s/5u4l67faq46jbvw/tamil_spm_8k.vocab
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:6018:1::a27d:301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/5u4l67faq46jbvw/tamil_spm_8k.vocab [following]
--2019-04-06 13:52:31--  https://www.dropbox.com/s/raw/5u4l67faq46jbvw/tamil_spm_8k.vocab
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4aea5a4e4b7104931435c22897.dl.dropboxusercontent.com/cd/0/inline/AeiMh4RkoKW_Gdlc2aytqpYmS0-c8Njg4lU1SscAj11jl6t3cbxS-JAsAMNwTZWmiqrk0nsTaA9FEKeauoF8STXiiKbq7eoAcPyBfhKKpoB4HA/file# [following]
--2019-04-06 13:52:32--  https://uc4aea5a4e4b7104931435c22897.dl.dropboxusercontent.com/cd/0/inline/AeiMh4RkoKW_Gdlc2aytqpYmS0-c8Njg4lU1SscAj11jl6t3cbxS-JAsAMNwTZWmiqrk0nsTaA9FEKeauoF8STXiiKbq7eoAcPyBfhKKpoB4HA/file
R

In [34]:
!wget https://www.dropbox.com/s/24nq1vefalhkjjv/inltk.zip

--2019-04-06 13:56:42--  https://www.dropbox.com/s/24nq1vefalhkjjv/inltk.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/24nq1vefalhkjjv/inltk.zip [following]
--2019-04-06 13:56:42--  https://www.dropbox.com/s/raw/24nq1vefalhkjjv/inltk.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc210b434dd1e502ae61e3e1c9cc.dl.dropboxusercontent.com/cd/0/inline/AehPNdVh3kKxu6unMBTJv63rDOytbKy7Rj9MbfLj3o144t4smujl13cLNfstlg9Qg4SDkexz9J88q7F1K-6h_uAtekrmFQZVZx7EhPvatN48Yw/file# [following]
--2019-04-06 13:56:42--  https://uc210b434dd1e502ae61e3e1c9cc.dl.dropboxusercontent.com/cd/0/inline/AehPNdVh3kKxu6unMBTJv63rDOytbKy7Rj9MbfLj3o144t4smujl13cLNfstlg9Qg4SDkexz9J88q7F1K-6h_uAtekrmFQZVZx7EhPvatN48Yw/file
Resolving uc210b434dd1e502ae

In [35]:
!unzip inltk.zip

Archive:  inltk.zip
   creating: inltk/
  inflating: inltk/.DS_Store         
   creating: __MACOSX/
   creating: __MACOSX/inltk/
  inflating: __MACOSX/inltk/._.DS_Store  
   creating: inltk/__pycache__/
  inflating: inltk/__pycache__/.DS_Store  
   creating: __MACOSX/inltk/__pycache__/
  inflating: __MACOSX/inltk/__pycache__/._.DS_Store  
  inflating: inltk/__pycache__/tokenizer.cpython-36.pyc  
 extracting: inltk/init.py           
  inflating: __MACOSX/inltk/._init.py  
  inflating: inltk/tokenizer.py      
  inflating: __MACOSX/inltk/._tokenizer.py  


In [36]:
!ls

data_lm_tawiki_8k.pkl  inltk.zip  sample_data	      tamil_spm_8k.vocab
inltk		       __MACOSX   tamil_spm_8k.model  wikitalm_8k_447_third.pth


In [43]:
!mkdir models
!mv wikitalm_8k_447_third.pth models/

mkdir: cannot create directory ‘models’: File exists
mv: cannot stat 'wikitalm_8k_447_third.pth': No such file or directory


### Step 3. Setup the tokenizer

In [0]:
torch.cuda.set_device(0)

In [0]:
path = Path('.')

In [0]:
class TamilTokenizer(BaseTokenizer):
    def __init__(self, lang:str):
        self.lang = lang
        self.sp = spm.SentencePieceProcessor()
        self.sp.Load(str("tamil_spm_8k.model"))
        
    def tokenizer(self, t:str) -> List[str]:
        return self.sp.EncodeAsPieces(t)

In [0]:
sp = spm.SentencePieceProcessor()
sp.Load(str('tamil_spm_8k.model'))
itos = [sp.IdToPiece(int(i)) for i in range(8000)]

In [0]:
tamil_vocab = Vocab(itos)

In [0]:
tokenizer = Tokenizer(tok_func=TamilTokenizer, lang='ta')

In [32]:
tokenizer.special_cases

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep']

### Step 4. Load the Data bunch into a fast ai object.

In [0]:
data_lm = load_data('.', 'data_lm_tawiki_8k.pkl')

In [38]:
data_lm.show_batch()

idx,text
0,"▁c ric k et <unk> ▁a s s o ci ation <unk> ▁s ta d ium ) ▁என்பது ▁ராஜ் கோட் , இந்தியா வில் ▁உள்ள ▁துடுப்பாட்ட ▁அரங்க ம் ▁ஆகும் . ▁இது ▁இது ▁காந்த ேரி ▁துடுப்பாட்ட ▁அரங்க ம் ▁எனவும் ▁ அழைக்கப்படுகிறது ) ▁ , ▁இது ▁குசராத் து ▁மாநிலத்தின் ▁முதல் ▁சூரிய ▁ஆற்றல் ▁அரங்க ம் ▁ஆகும் . ▁நவம்பர் ▁ , ▁2015 ▁இல் ▁இந்த ▁அரங்க ம் ▁உட்பட ▁இந்தியாவில் ▁உள்ள ▁ஆறு ▁புதிய ▁துடுப்பாட்ட ▁அரங்க ு கள்"
1,"▁ / ▁ wiki ? curi d = 21 08 22 "" ▁title ="" தமிழ் ▁மொழியில் ▁உள்ள ▁பேச்சு ▁வழக்கில் ▁திரி ந்த ▁தமிழ்ச் ▁சொற்கள் ""> ▁தமிழ் ▁மொழியில் ▁உள்ள ▁பேச்சு ▁வழக்கில் ▁திரி ந்த ▁தமிழ்ச் ▁சொற்கள் ▁தமிழ் ▁மொழியில் ▁உள்ள ▁சில ▁சொற்கள் ▁காலப்போக்கில் ▁பேச்சு ▁வழக்கில் ▁பொருள் ▁திரி ந்து ▁எழுத்துக்கள் ▁திரி ந்து ▁உள்ளன . ▁அவற்றின் ▁தொகுப்பு . ▁< ▁ / ▁ doc > ▁< doc ▁id ="" 21 08 23"" ▁ url ="" https :"
2,"சி க் மக ளூர் ""> ▁சி க் மக ளூர் ▁சி க் மக ளூர் ▁("" <unk> ch ick ma g al ur "") ▁இந்தியாவின் ▁கர்நாடகா ▁மாநிலத்தில் ▁சி க்க ம கள ூ ரு ▁மாவட்டத்தில் ▁அமைந்துள்ள ▁ஒரு ▁நகரம் . ▁குளம் பி க்கு ▁( co ff ee ) ▁புகழ்பெற்ற ▁மு ல்ல யா நகர ி ▁மலைத்தொடர் ▁அடி வார த்தில் ▁சி க் ம ங்கள ூர் ▁அமைந்துள்ளது , ▁இது ▁கர்நாடகா வின் ▁குளம் பி ▁விளை யும் ▁நில மாக"
3,"▁1 20 ▁மில்லி ▁ஆம் பியர் ▁வரையிலும் ▁உள்ளது . ▁இவ்விரு ▁மின்னோட்ட ங்களும் ▁தனித்தனி யான வை யான ாலும் ▁ஒன்ற ி ற்க ொன்று ▁தொடர்புடைய ன . ▁இவை களை ▁இணைக்கும் ▁ஒரு ▁காரணி ▁எதிர்மின் ▁முனை க்கரு கில் ▁காணப்படும் ▁இட ▁மின்ன ூட்ட மாகும் ▁("" s p ace ▁c har ge "") . ▁குழாய ில் ▁இரு மின் ▁முனை களுக்கு முள்ள ▁மின் ▁அழுத்த ▁வேறுபாடு ▁( p . d ) ▁குறைவாக ▁உள்ள போது , ▁இழை யிலிருந்து ▁இலத்திரன் கள் ▁வி"
4,ங்கள் ▁இடம்பெற்ற ுள்ள ▁ஒரு ▁சம் பிர தா ய ▁மத மாக ▁இருந்தது . ▁இதனால் ▁ஓவிய ம் ▁மற்றும் ▁சிற்பங்கள் ▁உள்ள ▁பட ங்கள் ▁உட்பட்ட ▁மத ப் ▁பொருட்களை ▁உருவாக்கும் ▁ஊக்க ம் ▁உ ண்ட ானது . ▁1930 ▁ களில் ▁ஸ்டா லி னின் ▁சீர்திருத்த க் ▁கருத்து களுக்குப் ▁பிறகு ▁பௌத்த ம் ▁மற்றும் ▁மத குரு ▁நம்பிக்கை கள் ▁மங்கோலிய ▁மக்கள் ▁குடியரச ில் ▁செல்வ ாக்க ிழந்த ன . ▁உட்புற ▁மங்கோலிய ாவில் ▁பாரம்பரிய ▁மத க் கொள் கை கள் ▁பாதிக்கப்பட்ட ு ▁கலாச்சார


In [39]:
len(data_lm.vocab.itos)

8000

### Step 5. Load pretrained model into a fastai object

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3, pretrained=False)

In [0]:
learn.load('wikitalm_8k_447_third', with_opt=True);

### Step 6. Generate sample text from learned model.

In [0]:
def create(text, spm=False, N_WORDS=50, N_SENTENCES=4):
    TEXT=sp.EncodeAsPieces(text)
    TEXT=' '.join(TEXT)
    sp_merge=lambda s:s.replace(' ','').replace('▁',' ')
    samples=[]
    for _ in range(N_SENTENCES):
      sample=learn.predict(TEXT, N_WORDS, temperature=0.7)
      samples.append(sample)
      if spm:
          print(sample)
          print()
    if spm:
        print()
    for sample in samples:
      sample=sp_merge(sample)
      print(sample)
      print()

In [46]:
text='17-ஆம் நூற்றாண்டில்'
create(text, spm=True)

▁17 - ஆம் ▁நூற்றாண்டில் ▁மற்றும் ▁ஆர் த ரிய ப் ▁போ ப்பா ண்ட வர் களை ▁ தற் செயல ாகக் ▁கண்டறிந்த ு ▁நேர் ந்த தாக ▁கருதப்படுகிறது . ▁இது ▁குறித்த ▁ஆய்வு களின் ம் ▁படி வளர்ச்சி க் ▁கால த்தை ▁அடிப்படையாகக் ▁கொண்ட தாகவும் , ▁குறிப்பாக , ▁புதிய ▁சமுதாய ம் ▁மற்றும் ▁புற ப் பரப்பு களின் ▁நடு வே ▁சமய ப் ▁பரப்பு

▁17 - ஆம் ▁நூற்றாண்டில் ▁உருவாக்கப்பட்ட து . ▁இந்த ▁ ரை சின் ▁குடும்ப த்தா ரால் ▁மேற்கொள்ள ப்படும் ▁ஒரு ▁புதிய ▁முயற்சி யானது ▁கடந்த ▁10 ▁ஆண்டுகளில் யிர ான் , ▁கால ▁நிறைவு க் ▁காலம் , ▁மற்றும் ▁முன் னோ க்கி ச் ▁சென்ற ுள்ளன ▁என்று ▁கூறப்படுகிறது . ▁வேறு ▁சில ▁பள்ளி களில் ▁இருந்து ம் , ▁பல ▁இடங்களுக்கு ▁ஒரு ▁தனி ச் சிறப்ப

▁17 - ஆம் ▁நூற்றாண்டில் , ▁" " v il le y " " ▁என்ற ▁பெயரில் ▁" " நான் ▁செய்ய க் ▁கூடிய ▁நீ ரி ழிவு " " ▁என்று ▁அதன் ▁பெயரை ▁மாற்றி க் ▁கொண்டு , ▁அதற்கு ▁' க ங் ரீ னா ' ▁எனப் ▁பெயரிட்ட ார் . ▁அது ▁“ அ ன் பு ▁நீ ங்கள் ” ▁என ▁குறிப்பிட

▁17 - ஆம் ▁நூற்றாண்டில் ▁” சா ரா ” , ▁“ இரு ம்பு ” , ▁' பே ரி ஸ் ' , ▁“ சூ ரியன் ” , ▁“ ஆ ல் ’ , ▁“ பி ரை யர் ” , ▁“ பி ரியா ஸ் ” , ▁” பூ ஸ

In [47]:
text='தனது ஆட்சிக் காலத்தில் பிரான்சின் அரசுத்தலைவர்'
create(text)

 தனது ஆட்சிக் காலத்தில் பிரான்சின் அரசுத்தலைவர், மன்னர் முதலியோர் இருந்தனர். < / doc> <doc id="212709" url="https: / / ta.wikipedia.org / wiki?curid=213209" title="வே

 தனது ஆட்சிக் காலத்தில் பிரான்சின் அரசுத்தலைவர் முர்ரே வின்சன்ட் மற்றும் அவரது மனைவி பிரவுன் ஆகியோரின் சகோதரன் சதியாவிடமிருந்து விடுதலை பெற்று, மன்சப்தாரியின் இடத்தைப் பிடிக்காமல் நடந்து கொள்ளவில்லை. ஆண்களை பெண்களுக்குக் கொண்டு சென்ற போது, மீண்டும் ஆணுறை

 தனது ஆட்சிக் காலத்தில் பிரான்சின் அரசுத்தலைவர் மன்ஹாட்டன் ட்ரெஷ்வாவை இடித்துவிட்டு, லைபீரியாவுக்கு வெளியே, லின்டாக், வெர்ஜின், ஸ்கைஸ்காடன், கோடாஸ்ட், ட்ரோலிங், லிட்டில் டேல்

 தனது ஆட்சிக் காலத்தில் பிரான்சின் அரசுத்தலைவர் எட்வர்டு மார்ட்டின் பிரபுவின் ஆட்சியின் கீழ் வந்தது. இதன் போது இந்த யுத்தத்தின் போது, பதினேழாம் நூற்றாண்டில், மானிடர்களின் வளர்ச்சியை தடுக்க பல திட்டங்களை வகுத்தது. இச்சபையின் நிறுவனர் கணிசமாக இருந்தார். இதன்



In [48]:
text='இன்று இவர் பிரான்ஸ், ஜேர்மனி ஆகிய'
text='இவரது ஆக்கங்கள் பல்வேறு'
create(text)

 இவரது ஆக்கங்கள் பல்வேறுபட்ட மூலக்கூற்றில் உள்ளடக்கப்பட்டுள்ளன. < / doc> <doc id="191495" url="https: / / ta.wikipedia.org / wiki?curid=191498" title

 இவரது ஆக்கங்கள் பல்வேறு பொருட்கள்,பட்டியல்கள், அணிகலன்கள், வெற்றிகிணறுகள், பல்வகையான வாகனங்கள், பண்ணைகள், பாறைகள் மற்றும் நகைகள் ஆகியவற்றைப் பயன்படுத்திக்கொண்டுன. இதுபோன்று, பருவம் மற்றும் வருடாந்திர நாணயங்கள்

 இவரது ஆக்கங்கள் பல்வேறுபட்ட விவரக் கூற்றுகளுடன் இணைக்கப்பட்டுள்ளன. அவை: < / doc> <doc id="191294" url="https: / / ta.wikipedia.org / wiki?curid=191294

 இவரது ஆக்கங்கள் பல்வேறு வகையிலும் உள்ளன. ஒருசிலரும் இவ்வளிமையில் தங்கள் உறவுகளை வெளிப்படுத்திக் கொள்கிறார்கள். இவர்களைக் கொண்டு அறியப்படவில்லை. இருந்தும் அவர்களுக்காக சவாலுக்குரிய மிக உயரிய விருதான பத்மசிறீ விருது வழங்கி சிறப்பிக்கப்படுகின்றது.



### Step 7. Prompt it to create entire article segments from scratch.

In [49]:
sp_merge=lambda s:s.replace(' ','').replace('▁',' ')
sp_merge('<doc id="91202" url="https: / / ta.wikipedia.org / wiki?')

'<docid="91202"url="https://ta.wikipedia.org/wiki?'

In [50]:
create('<doc id="52700" url="https://ta.wikipedia.org/wiki?', N_WORDS=200, N_SENTENCES=10)

 <doc id="52700" url="https://ta.wikipedia.org/wiki? www.twofa.com / tamil / article107904.html < / doc> <doc id="160883" url="https: / / ta.wikipedia.org / wiki?curid=160883" title="பொருளாதாரம்"> பொருளாதாரம் பொருளாதாரம் என்பது உற்பத்தி மற்றும் உற்பத்தியின் ஒரு உற்பத்தி ஆகும். அது உற்பத்தி, உற்பத்தி, உற்பத்தி, உற்பத்தி, உற்பத்தி, உற்பத்தி, மேலாண்மை, உற்பத்தி, விநியோகம் என பல துறைகளிலும் உற்பத்தி செய்யப்படுகின்றன. அடிப்படை உற்பத்தியாளர்களின் எண்ணிக்கை, உற்பத்தி, உற்பத்தி, உற்பத்தி, உற்பத்தி மற்றும் உற்பத்தி, உற்பத்தி, உற்பத்தி, உற்பத்தி, உற்பத்தி, நுகர்வோர் மற்றும் வேளாண்மை, உற்பத்தி, உற்பத்தி, தொழில்துறை, பானம் ஆகியவற்றைப் பயன்படுத்துகிறது. < / doc> <doc id="81413" url="https: / / ta.wikipedia.org / wiki?curid=82613" title

 <doc id="52700" url="https://ta.wikipedia.org/wiki? / http: / / www.news.com / national / films / 2016 / 08 / blog-post_10.html < / doc> <doc id="190598" url="https: / / ta.wikipedia.org / wiki?curid=190594" title="உயிரியல்"> உயிரியல் உயிரியல் (modology) என்பது, ஒர

 ### I am amazed:
 
 by these two sentences on chemical reaction: I verified that these are not directly lifted from Wikipedia. Neither are these sentences available on any google searched Tamil page.

ஒரு பொருள் வெப்பம் அடைந்து வேதிப்பொருளாக வினைபுரிகிறது. 
வினையின் இறுதி வினையில் வினையின் விளைவுகள் தோன்றுகின்றன.

## Obtaining Word vectors

Following code is adapted from: (https://github.com/cstorm125/thai2fit/blob/master/thwiki_lm/thwiki_lm.ipynb)

In [0]:
# Following code is adapted from: https://github.com/cstorm125/thai2fit/blob/master/thwiki_lm/thwiki_lm.ipynb

In [61]:
#how we extracted the embeddings
emb_weights = list(learn.model.named_parameters())[0][1]
emb_np = to_np(emb_weights.data)
emb_np.shape

(8000, 400)

In [0]:
import pandas as pd

In [63]:
tamilvec = pd.DataFrame(emb_np)
tamilvec.index = data_lm.vocab.itos
print(tamilvec.shape)
tamilvec.tail(10)

(8000, 400)


,0,1,2,3,4,5,6,7,8,9,...,390,391,392,393,394,395,396,397,398,399
▁இருபத்தி,-1.077665,-0.390356,0.163234,0.283401,-1.093594,0.503261,0.321981,0.401306,0.076390,0.727045,...,-0.325647,-0.522335,-0.188289,-0.356652,-0.485064,-0.121918,-0.449520,-0.116807,-0.113558,0.338918
▁சந்தர்ப்ப,-0.121100,-0.049716,-0.167197,0.365919,-1.635901,1.082761,-0.331799,-0.512262,-0.723081,0.198193,...,-0.447440,0.219738,-0.426128,-0.254355,0.416040,0.167318,-0.591591,0.540607,-0.062622,0.472127
சமூக,0.209300,0.599686,0.039264,0.364590,-1.840876,0.746350,0.312119,-0.538757,-1.021886,-0.038306,...,-0.784274,-0.125212,-0.364481,-0.409861,-1.035161,0.422461,-0.901599,0.352806,0.328277,0.911684
▁வற்புறுத்த,-0.426133,0.192765,0.032120,-0.392156,-1.239767,0.556953,0.268325,-0.056107,-1.168194,0.245597,...,-0.444652,-0.382560,-0.342674,-0.075639,-0.399160,0.054154,-0.710152,0.367050,0.494992,0.724223
▁கூகுள்,-0.485771,0.094991,0.122683,0.529538,-1.633889,0.722075,-0.180137,0.111167,-1.436899,0.640899,...,-0.305726,-0.358842,0.399187,-0.595666,-0.730073,-0.012862,-0.467213,0.846618,-0.061463,-0.000046
▁பிரசித்தி,-0.039581,-0.460073,0.109150,0.130076,-1.213040,0.653963,-0.115663,-0.748029,-1.023990,0.631861,...,-0.744943,-0.007287,0.159883,-0.335717,-1.102930,-0.095574,-0.667286,0.495898,0.475527,-0.187048
▁சீக்கியர்கள,-0.457573,-0.029164,0.267646,0.384505,-0.755809,0.871107,0.213756,0.196102,-1.500173,0.445593,...,-0.522237,0.404235,-0.183606,-0.075815,-0.504381,-0.235018,-0.947623,0.127727,0.167060,-0.604006
▁மத்திமம்,-0.537851,0.323388,-0.127235,0.247344,-1.732942,0.839073,-0.374221,-0.406411,-0.810076,0.052043,...,0.429243,-0.557794,-0.199908,-0.246132,-0.220002,0.181184,-0.230351,-0.076691,0.397272,0.882706
“,-0.487057,0.101350,0.323846,0.379314,-1.294675,0.587669,0.423591,-0.214962,-1.348075,0.608770,...,-0.735086,-0.604390,0.388983,-0.206811,-1.178709,-1.793508,-0.123695,0.008620,-0.169480,0.077503
‘,-1.062528,-0.326936,-0.282349,0.499380,-1.066751,0.758417,-0.177993,-0.499733,-1.039507,0.274381,...,-0.243882,-0.269090,0.277599,-0.375555,-1.238790,0.933866,0.082637,-0.540410,0.465956,0.394246


In [0]:
tamilvec.to_csv('tamilvec_wiki8k.csv')

## Exploring Word vectors

In [0]:
word2index = tamilvec.index
tam2vec = dict(zip(tamilvec.index, emb_np))


In [117]:
tam2vec['▁பல்கலைக்கழகம்']

array([-0.203724,  0.020787,  0.32179 , -0.130676, ..., -0.058546, -0.193656,  0.322165, -0.799783], dtype=float32)

In [0]:
from collections import Counter
import math 

def similar(target='▁பல்கலைக்கழகம்', N=15):
    scores = Counter()
    for word in word2index:
        raw_difference = tam2vec[word] - (tam2vec[target])
        squared_difference = raw_difference * raw_difference
        scores[word] = -math.sqrt(sum(squared_difference))

    return scores.most_common(N)

In [119]:
similar('▁பல்கலைக்கழகம்')

[('▁பல்கலைக்கழகம்', -0.0),
 ('▁பல்கலைக்கழக', -7.622758631724941),
 ('▁பல்கலைக்', -8.381474130733048),
 ('▁அருங்காட்சியகம்', -9.934804107363593),
 ('▁கல்லூரி', -10.041136493606821),
 ('▁நூலகம்', -10.318009302485999),
 ('▁பாடசாலை', -10.527922100302066),
 ('▁பள்ளி', -10.59030479469354),
 ('▁உயர்நிலைப்பள்ளி', -10.687967829421753),
 ('▁கழகம்', -10.696278333915997),
 ('▁நிறுவனம்', -10.705079108673019),
 ('▁நூலக', -10.819976809333246),
 ('▁பேரரசு', -10.837515571472192),
 ('▁பொறியியல்', -10.891923176733965),
 ('▁தேவாலய', -10.900616360515421)]

In [0]:
import random
def random_word_similarity():
  word = random.choice(word2index)
  print('Words most similar to:', word, 'are:')
  for w,d in similar(word):
    print(w,d)

In [121]:
random_word_similarity()

Words most similar to: ▁பொதுமக்கள் are:
▁பொதுமக்கள் -0.0
▁பொதுமக்கள -7.311791132445025
▁தனிநபர் -8.11006542331366
▁பார்வையாளர் -8.458276101922129
▁ஆயிரக்கணக்கான -8.805056662231298
▁தன்னார்வ -8.930435492939377
▁சுற்றுச்சூழல் -8.939269502319872
▁தொழில்துறை -9.029742338632035
▁வாடிக்கையாளர் -9.057488201825205
▁ஊழல் -9.105347385604688
▁முற்றிலும் -9.13567447780089
▁ஏழை -9.154276349677357
▁வாடகை -9.16692868653695
▁இரகசிய -9.18638676898657
▁இளைஞர் -9.19533455846729


In [122]:
random_word_similarity()

Words most similar to: த்தாக்க are:
த்தாக்க -0.0
▁இறுக்க -12.691843130267445
▁சூத்திர -12.721162441701232
▁உண்டாக்க -12.737521652437568
▁வரைபட -12.768830127707032
▁உருமாற்ற -12.782229213630288
▁வெளிப்பாடு -12.82524256073423
▁எந்திர -12.840977995027243
="120 -12.84932002275394
▁அளவீடு -12.88993905821583
▁முதலீட்ட -12.917301073560777
▁பலவீன -12.917775157007249
▁நுணுக்க -12.917878760376594
க்கோள -12.92487661726165
="364 -12.92950152204314


In [123]:
for _ in range(10):
  random_word_similarity()
  print()

Words most similar to: ▁இங்கிலாந்தில் are:
▁இங்கிலாந்தில் -0.0
▁இங்கிலாந்தின் -7.315344645966783
▁இங்கிலாந்த -7.576312923061142
▁ஐரோப்பாவில் -7.669542609285551
▁ஸ்பெயின -8.136127158267897
▁நெதர்லாந்த -8.222772243618472
▁சீனாவில் -8.260297278307958
▁அயர்லாந்து -8.39196202988463
▁பிரிட்டன -8.410181398580907
▁பிரான்சின் -8.434232077852023
▁சுவிட்சர்லாந்த -8.448634197637523
▁சிகாகோ -8.485547406671344
▁டென்மார்க் -8.486482326598376
▁லண்டன் -8.495074259212648
▁நோர்வே -8.499953224843622

Words most similar to: மடைந்த are:
மடைந்த -0.0
முற்ற -10.445327544531828
மடை -10.70328315345914
ங்களுடன் -11.65593482115408
மூட்ட -11.862272778139955
த்துடன் -11.912952068325405
டைந்த -11.95720243513975
ப்படுத்தப்பட்ட -11.974903330243833
="270 -12.016728299186642
த்திற்குப் -12.047011655999412
▁அடைந்த -12.071368862900227
ப்படுத்திய -12.09185544693538
ப்பெற்ற -12.192605321079991
▁மேம்பட்ட -12.230622755358013
ங்களுக்கு -12.297557645367915

Words most similar to: ப்பின் are:
ப்பின் -0.0
ப்பில் -8.952442785370923

In [0]:
def ten_random_words():
  for _ in range(10):
    random_word_similarity()
    print()

In [158]:
ten_random_words()

Words most similar to: ாலும் are:
ாலும் -0.0
ினாலும் -11.50416925436963
ால் -12.238528245754344
ாலேயே -12.392875786454107
ிருந்தாலும் -12.55110066983433
ுடனும் -12.58962048722541
ிடமிருந்து -12.662696338813307
="106 -12.802532662734464
%25 -12.845250415562589
="270 -12.895210523172317
தோடு -12.909532446020693
▁போதும் -12.926834938567456
ோடு -13.005850698323137
ுடனான -13.05179008804898
="349 -13.054159724766237

Words most similar to: and are:
and -0.0
amb -11.782760418899533
ank -12.066623676561559
app -12.16313600515206
amp -12.203073412284358
aw -12.311364281336406
att -12.363953684631184
ara -12.481839214558365
ava -12.54877433647053
ant -12.595197248628269
ath -12.623819692007451
eg -12.643674736513232
ag -12.666560981819856
ak -12.73685297475567
ad -12.773424811643851

Words most similar to: ▁குற்ற are:
▁குற்ற -0.0
="270 -8.779132853743242
▁H -8.997777965496361
Ar -8.998914133287355
CO -8.99912050869038
▁Pro -8.999210347926848
F -8.999330251534237
National -8.999435536651761
▁S -8

In [126]:
similar('▁ராமசாமி')

[('▁ராமசாமி', -0.0),
 ('▁சந்திரசேகர', -9.54532195374704),
 ('▁மோகன்', -10.401543462473082),
 ('▁லட்சுமி', -10.41289789273595),
 ('▁கணபதி', -10.480418163663533),
 ('▁சரஸ்வதி', -10.516065672799293),
 ('▁கார்த்திக்', -10.773850896618205),
 ('▁நடராஜ', -10.779588982132745),
 ('▁சரவண', -10.789375278462977),
 ('▁மகாதேவ', -10.835675146767029),
 ('▁விஸ்வநாத', -10.888252048870958),
 ('▁ராதா', -10.889131102231273),
 ('▁சுப்பிரமணிய', -10.902323328384806),
 ('▁சாஸ்தா', -10.943302353859444),
 ('▁ஜோசப்', -10.966607630312273)]

In [127]:
similar('▁ஜோசப்')

[('▁ஜோசப்', -0.0),
 ('▁டேவிட்', -7.779510261975913),
 ('▁ராபர்ட்', -7.8284473989000976),
 ('▁அந்தோனி', -7.841824199835671),
 ('▁மைக்கேல்', -7.858238270135278),
 ('▁தாமஸ்', -8.02338192402103),
 ('▁சார்லஸ்', -8.264686424530728),
 ('▁ஜேம்ஸ்', -8.32321120202982),
 ('▁லூயி', -8.471510423365268),
 ('▁ஹென்றி', -8.491007852703259),
 ('▁பிரான்சிஸ்', -8.613287537556847),
 ('▁ரிச்சர்ட்', -8.757825038339764),
 ('▁மேரி', -8.782933898887592),
 ('▁ஜார்ஜ்', -8.799716191733692),
 ('▁ஸ்காட்', -8.873270538462407)]

In [128]:
similar('▁சரஸ்வதி')

[('▁சரஸ்வதி', -0.0),
 ('▁பார்வதி', -8.901737269767553),
 ('▁லட்சுமி', -9.022779847792453),
 ('▁பகவதி', -9.254674648756799),
 ('▁விஷ்ணு', -9.256148167600216),
 ('▁பவானி', -9.444592772933724),
 ('▁துர்க்கை', -9.494334391964024),
 ('▁காவேரி', -9.509799344996821),
 ('▁கிருஷ்ணா', -9.561535599285317),
 ('▁கணபதி', -9.607123751864426),
 ('▁ஹரி', -9.608417053612943),
 ('▁சமசுகிருத', -9.614878965722438),
 ('▁காளி', -9.618903142611305),
 ('▁பிரம்ம', -9.738397167370227),
 ('▁மைசூர்', -9.776696960662528)]

In [129]:
similar('▁கன்னியாகுமரி')

[('▁கன்னியாகுமரி', -0.0),
 ('▁திருநெல்வேலி', -7.527455691819665),
 ('▁இராமநாதபுரம்', -7.573070251247375),
 ('▁தூத்துக்குடி', -7.949094952619637),
 ('▁கோயம்புத்தூர்', -8.042168855819247),
 ('▁கடலூர்', -8.151464592979943),
 ('▁புதுக்கோட்டை', -8.167071993045594),
 ('▁ஈரோடு', -8.19541341881066),
 ('▁விருதுநகர்', -8.271042453532667),
 ('▁நீலகிரி', -8.291753518733088),
 ('▁திருப்பூர்', -8.371182704832854),
 ('▁சேலம்', -8.389907388885343),
 ('▁திண்டுக்கல்', -8.437733686418069),
 ('▁விழுப்புரம்', -8.583561505773364),
 ('▁கர்நாடகா', -8.586033472059603)]

In [130]:
similar('▁கர்நாடகா')

[('▁கர்நாடகா', -0.0),
 ('▁கேரளா', -6.157619228265201),
 ('▁மகாராஷ்டிரா', -6.4709969603943165),
 ('▁கர்நாடக', -6.587882040414094),
 ('▁ஒடிசா', -7.376039538781334),
 ('▁மகாராட்டிர', -7.556998792598648),
 ('▁கோவா', -7.895222022448399),
 ('▁அரியானா', -7.905754684586284),
 ('▁குசராத்', -7.917046637531333),
 ('▁புதுச்சேரி', -8.001225127736078),
 ('▁கேரள', -8.072821314444006),
 ('▁பெங்களூரு', -8.082008293857768),
 ('▁குஜராத்', -8.106104624638144),
 ('▁ஆந்திர', -8.31719240784472),
 ('▁ராஜஸ்தான்', -8.335459715579125)]

In [131]:
random_word_similarity()

Words most similar to: ▁ஐய are:
▁ஐய -0.0
▁அய்ய -8.85300473261025
▁சந்தேக -9.601295265937896
="106 -9.874883438438179
="270 -9.899027460057466
%25 -9.900223235034074
="349 -9.91724822634812
="367 -10.064806074542847
▁அறிவ -10.149942055816046
▁ஆழ்ந்த -10.160315241117678
▁உரோம -10.191083869757493
▁இன்னும் -10.242025476213183
▁ஆன்மீக -10.244959156704075
▁போலி -10.256480477912707
▁ஐரோப்பிய -10.328563584407522


In [156]:
ten_random_words()

Words most similar to: யோடு are:
யோடு -0.0
யுடன் -8.840733505209863
யினால் -9.767475028085022
யுடனும் -9.934276982380087
யால் -10.174591894505156
யினை -10.270553361071448
யைத் -10.421312241628822
யைப் -10.530115057662075
யையும் -10.643601224399173
யிடம் -10.925329371555097
களோடு -10.96884680800126
யிலே -10.969741695852
யைக் -10.974313024690337
யிலிருந்து -11.026692742798437
யாய் -11.042025830231387

Words most similar to: her are:
her -0.0
he -10.97578164330279
ass -12.105565453824813
ho -12.228109966435456
oph -12.292087927469682
hi -12.426776675881154
tr -12.50534397595288
tra -12.627115721865525
ound -12.673961877094573
we -12.67490916184979
gen -12.67823949117384
pe -12.757655889576233
mo -12.760798189885659
han -12.787713542065356
be -12.798593135041278

Words most similar to: ▁கற் are:
▁கற் -0.0
▁பாறை -9.287481439352007
▁பற் -9.413664446119645
▁களிமண் -9.502272436968873
▁மண் -9.55597259128389
▁பொற் -9.563502967113008
▁கற்ப -9.613873018350544
="270 -9.753088517543748
▁கல் -9.81076

In [157]:
ten_random_words()

Words most similar to: ▁முடிவுகள் are:
▁முடிவுகள் -0.0
▁நடவடிக்கைகள் -9.55479623668584
▁மதிப்பீடு -10.038379673108986
▁ஆய்வுகள் -10.068495110166985
▁புள்ளிவிவர -10.175114923449723
▁தகவல்கள் -10.21661680073663
▁கூற்று -10.242190364950229
▁செய்முறை -10.269941407897328
▁கருதுகோள -10.360168177963885
▁விவாத -10.40005274468901
▁விதிமுறை -10.413624694577827
▁கண்டுபிடிப்பு -10.430034884327988
▁விளைவுகளை -10.47917922290925
▁அளவீடு -10.530027455765275
▁படிம -10.570232000045548

Words most similar to: ▁திட்ட are:
▁திட்ட -0.0
▁திட்டம் -8.851953956821927
="270 -9.126066229250076
=124 -9.367733706146899
▁E -9.370397711434208
IP -9.370959304517568
National -9.37122732556526
▁H -9.37141178328729
D -9.371747793569023
John -9.371939015915508
SO -9.37194994202661
▁Ch -9.37223430499589
<s> -9.37226530040868
II -9.372288960069207
▁Pro -9.37236940649341

Words most similar to: ▁28 are:
▁28 -0.0
▁27 -4.176051140868481
▁29 -4.379672965706315
▁26 -4.41276755686684
▁25 -5.386994941155828
▁31 -5.414353377474656


In [170]:
similar('▁சிறந்த')

[('▁சிறந்த', -0.0),
 ('▁மிகச்சிறந்த', -9.458921686357114),
 ('▁சிறப்பான', -9.481669685361945),
 ('▁சிறப்பு', -9.54751757206307),
 ('▁மிகவும்', -9.631494487408778),
 ('▁முக்கியமான', -9.642608511865244),
 ('▁மிக', -9.742925606673618),
 ('▁சரியான', -9.751300920568529),
 ('▁நல்ல', -9.810143624373481),
 ('▁புகழ்பெற்ற', -9.829707228346788),
 ('▁முதன்மையான', -9.906328622143203),
 ('▁மிகப்பெரிய', -9.953689728206163),
 ('=124', -9.976620269960195),
 ('▁மிகச்', -10.015156365847014),
 ('▁அதிக', -10.018205591905877)]

## Analogy (To be completed)
Code for this function adapted from https://github.com/iamtrask/Grokking-Deep-Learning/blob/master/Chapter11%20-%20Intro%20to%20Word%20Embeddings%20-%20Neural%20Networks%20that%20Understand%20Language.ipynb

In [0]:
norms = np.sum(emb_np * emb_np,axis=1)
norms.resize(norms.shape[0],1)
normed_weights = emb_np * 1 #norms

def analogy(set1=['terrible','good'],set2=['bad']):
    query_vect = np.zeros(len(emb_np[0]))
    query_vect = normed_weights[word2index.get_loc(set1[0])] + normed_weights[word2index.get_loc(set1[1])] 
    query_vect -= normed_weights[word2index.get_loc(set2[0])]
    
    scores = Counter()
    for word in word2index:
        raw_difference = tam2vec[word] - query_vect
        squared_difference = raw_difference * raw_difference
        scores[word] = -math.sqrt(sum(squared_difference))
        
    return scores.most_common(10)[1:]

In [167]:
analogy(set1=['▁அற்புத', '▁நல்ல'], set2=['▁துல்லிய'])
## needs some more work.

[('▁அற்புத', -11.066252350065602),
 ('▁அழகு', -12.761643243976454),
 ('▁மிகச்சிறந்த', -12.788050229467792),
 ('▁அழகான', -12.83636766203465),
 ('▁அரிய', -12.840842443098275),
 ('▁அழகிய', -12.873413857744971),
 ('▁சிறப்பான', -12.92263649782561),
 ('▁சிறந்த', -13.022609914288727),
 ('▁புதுமை', -13.051496786361945)]